# 2. Feature Pipeline

## 2.1. Setup

### 2.1.1. Import Libraries and Initialize Hopsworks Connection

In [1]:
# Standard imports
import os
import sys
import json
import time
from datetime import date, datetime, timedelta
import warnings
from pathlib import Path
warnings.filterwarnings("ignore", module="IPython")

#  Establish project root directory
def find_project_root(start: Path):
    for parent in [start] + list(start.parents):
        if (parent / "pyproject.toml").exists():
            return parent
    return start

root_dir = find_project_root(Path().absolute())
print("Project root dir:", root_dir)

if str(root_dir) not in sys.path:
    sys.path.append(str(root_dir))

# Third-party imports
import requests
import pandas as pd
import great_expectations as gx
import hopsworks
from urllib3.exceptions import ProtocolError  
from requests.exceptions import ConnectionError, Timeout
from confluent_kafka import KafkaException
import numpy as np

#  Project imports
from utils import cleaning, config, feature_engineering, fetchers, hopsworks_admin, incremental, metadata

#  Load settings 
settings = config.HopsworksSettings()
HOPSWORKS_API_KEY = settings.HOPSWORKS_API_KEY.get_secret_value()
GITHUB_USERNAME = settings.GH_USERNAME.get_secret_value()

# Login to Hopsworks
project = hopsworks.login(api_key_value=HOPSWORKS_API_KEY)
fs = project.get_feature_store()

Project root dir: c:\Users\krist\Documents\GitHub\pm25
HopsworksSettings initialized!
2026-01-19 11:01:13,378 INFO: Initializing external client
2026-01-19 11:01:13,378 INFO: Base URL: https://c.app.hopsworks.ai:443
2026-01-19 11:01:15,408 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1279184
2026-01-19 11:01:17,175 INFO: Closing external client and cleaning up certificates.
Connection closed.
2026-01-19 11:01:17,181 INFO: Initializing external client
2026-01-19 11:01:17,181 INFO: Base URL: https://c.app.hopsworks.ai:443
2026-01-19 11:01:18,896 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1279184


### 2.1.2. Repository management

In [2]:
repo_dir = hopsworks_admin.clone_or_update_repo(GITHUB_USERNAME)
os.chdir(repo_dir)

Repository exists at c:\Users\krist\Documents\GitHub\pm25\notebooks\pm25-forecast-openmeteo-aqicn


### 2.1.3. Configure API Keys and Secrets

In [3]:
today = date.today()

if settings.AQICN_API_KEY is None:
    print("AQICN_API_KEY missing.")
    sys.exit(1)

AQICN_API_KEY = settings.AQICN_API_KEY.get_secret_value()

secrets = hopsworks.get_secrets_api()
try:
    secret = secrets.get_secret("AQICN_API_KEY")
    if secret is not None:
        secret.delete()
except Exception:
    pass

secrets.create_secret("AQICN_API_KEY", AQICN_API_KEY)

Secret created successfully, explore it at https://c.app.hopsworks.ai:443/account/secrets


Secret('AQICN_API_KEY', 'PRIVATE')

## 2.2. Get Feature Groups

In [4]:
air_quality_fg, weather_fg = hopsworks_admin.create_feature_groups(fs)

## 2.3. Load Metadata from Feature Group

In [5]:
# Load metadata from air_quality feature group
aq_data = air_quality_fg.read()

if len(aq_data) == 0:
    print("⚠️ No air quality data found. Run pipeline 1 (backfill) first.")
    sys.exit(1)

# Extract unique sensor metadata
metadata_df = aq_data[["sensor_id", "latitude", "longitude", "city", "street", "country", "aqicn_url"]].drop_duplicates(subset=["sensor_id"])
print(f"📍 Loaded metadata for {len(metadata_df)} sensors")
metadata_df = metadata_df.set_index("sensor_id")

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (13.04s) 
📍 Loaded metadata for 103 sensors


## 2.4. Data Collection
Loop through all sensors to fetch today's air quality data and weather forecasts, format data to match feature group schemas.

### 2.4.1. Initialize Processing Counters and Metadata

In [6]:
# Load metadata from feature group for nearby sensor calculations
metadata_indexed = metadata_df.copy()
metadata_indexed.index = metadata_indexed.index.astype(int)

# aq_successful = 0
# aq_failed = 0
# aq_skipped = 0

# weather_successful = 0
# weather_failed = 0
# weather_skipped = 0

print(f"🔍 Processing {len(metadata_indexed)} sensor locations.")

🔍 Processing 103 sensor locations.


### 2.4.2. Load Historical Air Quality Data (Last 4 Days)

In [7]:
historical_start = today - timedelta(days=4)
try:
    historical_df = air_quality_fg.read()
    if not historical_df.empty:
        historical_df["date"] = pd.to_datetime(historical_df["date"]).dt.tz_localize(None)
        today_dt = pd.to_datetime(today)
        historical_start_dt = pd.to_datetime(historical_start)
        
        # Include TODAY in historical data (we'll filter it out later per sensor)
        historical_df = historical_df[
            (historical_df["date"] >= historical_start_dt) & 
            (historical_df["date"] <= today_dt)  # Changed < to <=
        ][["date", "sensor_id", "pm25"]]
        
        historical_df = historical_df[historical_df["sensor_id"].isin(metadata_indexed.index)]
    else:
        historical_df = pd.DataFrame()
except Exception as e:
    print(f"⚠️ Error reading historical data: {e}")
    historical_df = pd.DataFrame()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (8.78s) 


### 2.4.3. Identify Missing Dates for Backfill

In [8]:
existing_dates = air_quality_fg.read()["date"].dt.date.unique()

today = datetime.today().date()
start_date = today - timedelta(days=7)  # Check last 7 days for missing data

expected_dates = pd.date_range(start=start_date, end=today, freq="D").date
missing_dates = [d for d in expected_dates if d not in existing_dates]

# print(f"📅 Missing dates to backfill: {missing_dates}")
formatted = ", ".join(d.isoformat() for d in missing_dates)
print(f"📅 Missing dates to backfill: {formatted}")



Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (6.60s) 
📅 Missing dates to backfill: 2026-01-15, 2026-01-16, 2026-01-17, 2026-01-18, 2026-01-19


In [9]:
# # Initialize containers for results
# aq_list = []
# weather_dict = {}  # sensor_id -> weather_df

# # Determine missing dates
# existing_dates = air_quality_fg.read()["date"].dt.date.unique()

# today = datetime.today().date()
# start_date = today - timedelta(days=7)  # or however far back you want to check

# expected_dates = pd.date_range(start=start_date, end=today, freq="D").date
# missing_dates = [d for d in expected_dates if d not in existing_dates]

### 2.4.4. Prepare Historical Data Window

In [10]:
historical_cutoff = pd.to_datetime(min(missing_dates)) - pd.Timedelta(days=3)
historical = air_quality_fg.read()
historical["date"] = pd.to_datetime(historical["date"]).dt.tz_localize(None)
historical = historical [historical["date"] >= historical_cutoff]

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (12.32s) 


### 2.4.5. Track Existing Sensor-Date Pairs

In [11]:
existing = historical[["sensor_id", "date"]].copy()
existing["date_only"] = existing["date"].dt.date
existing_keys = set(zip(existing["sensor_id"], existing["date_only"]))

### 2.4.6. Initialize Data Containers

In [12]:
all_aq_rows = [historical]
all_weather_rows = []

### 2.4.7. Fetch Missing Air Quality Data

In [ ]:
count = 1
for sensor_id, meta in metadata_df.iterrows():
    print(f"Fetching air quality for sensor {sensor_id}, {count}/{len(metadata_df)}")
    count += 1
    for day in missing_dates:
        # Skip any sensor date combination that already exists
        if (sensor_id, day) in existing_keys:
            continue
        try:
            aq_df = fetchers.get_pm25(
                meta["aqicn_url"], meta["country"], meta["city"],
                meta["street"], day, AQICN_API_KEY
            )
            if aq_df.empty or aq_df["pm25"].isna().all():
                continue

            aq_df["sensor_id"] = int(sensor_id)
            aq_df["pm25"] = pd.to_numeric(aq_df["pm25"], errors="coerce")
            aq_df["date"] = pd.to_datetime(aq_df["date"]).dt.tz_localize(None)
            
            # Add metadata columns
            aq_df["city"] = meta["city"]
            aq_df["street"] = meta["street"]
            aq_df["country"] = meta["country"]
            aq_df["aqicn_url"] = meta["aqicn_url"]
            aq_df["latitude"] = meta["latitude"]
            aq_df["longitude"] = meta["longitude"]
            
            aq_df = aq_df.drop(columns=["url"], errors="ignore")

            all_aq_rows.append(aq_df)

        except Exception as e:
            print(f"❌ Sensor {sensor_id} on {day}: {type(e).__name__}")
            continue

print(f"📊 Collected {len(all_aq_rows)} air quality dataframes")

Fetching air quality for sensor 60853, 1/103
Fetching air quality for sensor 59497, 2/103
Fetching air quality for sensor 59650, 3/103
Fetching air quality for sensor 112672, 4/103
Fetching air quality for sensor 60889, 5/103
Fetching air quality for sensor 60076, 6/103
Fetching air quality for sensor 58921, 7/103
Fetching air quality for sensor 84085, 8/103
Fetching air quality for sensor 89584, 9/103
Fetching air quality for sensor 198559, 10/103
Fetching air quality for sensor 149242, 11/103
Fetching air quality for sensor 105325, 12/103
Fetching air quality for sensor 78529, 13/103
Fetching air quality for sensor 88876, 14/103
Fetching air quality for sensor 65272, 15/103
Fetching air quality for sensor 77488, 16/103
Fetching air quality for sensor 351115, 17/103
Fetching air quality for sensor 122302, 18/103
Fetching air quality for sensor 196735, 19/103
Fetching air quality for sensor 69724, 20/103
Fetching air quality for sensor 60859, 21/103
Fetching air quality for sensor 6514

### 2.4.8. Fetch Missing Weather Forecast Data

In [29]:
count = 1
for sensor_id, meta in metadata_df.iterrows():
    print(f"Fetching weather for sensor {sensor_id}, {count}/{len(metadata_df)}")
    count += 1
    
    for day in missing_dates:
        try:
            # Fetch 7-day weather forecast starting from the missing date
            weather_df = fetchers.get_weather_forecast(
                sensor_id=sensor_id,
                latitude=meta["latitude"],
                longitude=meta["longitude"],
                start_date=day,
                end_date=day + timedelta(days=6)
            )
            
            if weather_df.empty:
                continue
            
            weather_df["sensor_id"] = int(sensor_id)
            weather_df["date"] = pd.to_datetime(weather_df["date"]).dt.normalize().dt.tz_localize(None)
            
            all_weather_rows.append(weather_df)
            
        except Exception as e:
            print(f"❌ Weather for sensor {sensor_id} on {day}: {type(e).__name__}")
            continue

print(f"📊 Collected {len(all_weather_rows)} weather dataframes")

Fetching weather for sensor 60853, 1/103
Fetching weather for sensor 59497, 2/103
Fetching weather for sensor 59650, 3/103
Fetching weather for sensor 112672, 4/103
2026-01-19 11:25:19,738 WARNING: Retrying (Retry(total=4, connect=5, read=4, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='api.open-meteo.com', port=443): Read timed out. (read timeout=None)")': /v1/forecast?latitude=57.70716&longitude=11.96679&start_date=2026-01-16&end_date=2026-01-22&daily=temperature_2m_mean&daily=precipitation_sum&daily=wind_speed_10m_max&daily=wind_direction_10m_dominant&timezone=UTC&format=flatbuffers
Fetching weather for sensor 60889, 5/103
Fetching weather for sensor 60076, 6/103
Fetching weather for sensor 58921, 7/103
Fetching weather for sensor 84085, 8/103
Fetching weather for sensor 89584, 9/103
Fetching weather for sensor 198559, 10/103
Fetching weather for sensor 149242, 11/103
Fetching weather for sensor 105325, 12/103
Fetching weather f

### 2.4.9. Clean and Align Data Structure

In [30]:
cleaned_aq_rows = []
expected_cols = historical.columns.tolist()

for i, df in enumerate(all_aq_rows):
    if df.empty or "pm25" not in df.columns or df["pm25"].isna().all():
        print(f"⚠️ Skipping empty or invalid df[{i}]")
        continue

    df["date"] = pd.to_datetime(df["date"]).dt.tz_localize(None)

    # Skip if too few expected columns are present
    if len(set(df.columns) & set(expected_cols)) < 3:
        print(f"⚠️ Skipping malformed df[{i}] with columns: {list(df.columns)}")
        continue

    # Align columns
    aligned = df.reindex(columns=expected_cols, fill_value=np.nan)

    # Final sanity check
    if aligned.shape[1] != len(expected_cols):
        print(f"❌ Still malformed after alignment: df[{i}] shape={aligned.shape}")
        continue

    # Force dtype alignment to match historical
    for col in expected_cols:
        if col in historical.columns:
            try:
                aligned[col] = aligned[col].astype(historical[col].dtype, errors="raise")
            except Exception as e:
                print(f"⚠️ Could not cast column '{col}' in df[{i}]: {e}")
                continue

    cleaned_aq_rows.append(aligned)

# Verify that column names and dtypes match
print("📋 Column names match:", all(df.columns.equals(historical.columns) for df in cleaned_aq_rows))

no_mismatch = True
for i, df in enumerate(cleaned_aq_rows):
    mismatched = [(col, df[col].dtype, historical[col].dtype)
                  for col in df.columns if col in historical.columns and df[col].dtype != historical[col].dtype]
    if mismatched:
        print("📋 Dtype mismatch:")
        print(f"  df[{i}] mismatches: {mismatched}")
        no_mismatch = False
if no_mismatch:
    print("📋 All dtypes match historical data.")
else:
    print("⚠️ Some dtypes do not match historical data.")

📋 Column names match: True
📋 All dtypes match historical data.


### 2.4.10. Combine and Clean Weather Data

In [31]:
if all_weather_rows:
    all_weather = pd.concat(all_weather_rows, ignore_index=True)
    all_weather = all_weather.sort_values(["sensor_id", "date"]).reset_index(drop=True)
    all_weather["date"] = pd.to_datetime(all_weather["date"]).dt.tz_localize(None)
    
    # Remove duplicates (same sensor, same forecast date)
    all_weather = all_weather.drop_duplicates(subset=["sensor_id", "date"], keep="first")
    
    print(f"🌤️ Total weather records: {len(all_weather)}")
    print(f"📅 Weather date range: {all_weather['date'].min()} to {all_weather['date'].max()}")
else:
    all_weather = pd.DataFrame()
    print("⚠️ No weather data collected")

🌤️ Total weather records: 1133
📅 Weather date range: 2026-01-15 00:00:00 to 2026-01-25 00:00:00


## 2.5. Feature Engineering

### 2.5.1. Combine Data and Add Engineered Features

In [32]:
# Combine data
all_aq = pd.concat([historical, *cleaned_aq_rows], ignore_index=True)
all_aq = all_aq.sort_values(["sensor_id", "date"]).reset_index(drop=True)
all_aq["date"] = pd.to_datetime(all_aq["date"]).dt.tz_localize(None)

# Add engineered features
all_aq = feature_engineering.add_rolling_window_feature(all_aq, window_days=3)
all_aq = feature_engineering.add_lagged_features(all_aq, lags=[1, 2, 3])
metadata_indexed = metadata_indexed.reset_index()
all_aq = feature_engineering.add_nearby_sensor_feature(all_aq, metadata_indexed, n_closest=3)

## 2.6. Insert Data to Feature Groups

### 2.6.1. Batch Insert Air Quality Data by Date

In [33]:
for day in missing_dates:
    day_rows = all_aq[all_aq["date"].dt.date == day].copy()
    day_rows = day_rows.dropna(subset=["pm25"])

    engineered_cols = [c for c in day_rows.columns if "lag" in c or "rolling" in c or "nearby" in c]
    day_rows = day_rows.dropna(subset=engineered_cols, how="any")

    if not day_rows.empty:
        # Convert types to match feature group schema
        day_rows = day_rows.astype({
            "sensor_id": "int32",
            "pm25": "float64",
            "pm25_lag_1d": "float64",
            "pm25_lag_2d": "float64",
            "pm25_lag_3d": "float64",
            "pm25_rolling_3d": "float64",
            "pm25_nearby_avg": "float64",
            "city": "string",
            "street": "string",
            "country": "string",
            "aqicn_url": "string",
            "latitude": "float64",
            "longitude": "float64",
        })
        
        # Ensure correct column order
        fg_columns = [f.name for f in air_quality_fg.features]
        day_rows = day_rows[fg_columns]
        
        air_quality_fg.insert(day_rows)
        print(f"✅ Inserted {len(day_rows)} rows for {day}")
    else:
        print(f"⚠️ No valid rows for {day}")

⚠️ No valid rows for 2026-01-15
2026-01-19 11:41:52,771 INFO: 	8 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279184/fs/1265800/fg/1952082


Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 00:01 | Remaining Time: 00:00


2026-01-19 11:42:02,274 WARNING: UserWarning: Materialization job is already running, aborting new execution.Please wait for the current execution to finish before triggering a new one.You can check the status of the current execution using `fg.materialization_job.get_state()`.or `fg.materialization_job.get_final_state()` or check it out in the Hopsworks UI.at https://c.app.hopsworks.ai:443/p/1279184/jobs/named/air_quality_1_offline_fg_materialization.
Use fg.materialization_job.run(args=-op offline_fg_materialization -path hdfs:///Projects/kristina_titanic/Resources/jobs/air_quality_1_offline_fg_materialization/config_1768459798660) to trigger the materialization job again.

✅ Inserted 1 rows for 2026-01-16
2026-01-19 11:42:02,608 INFO: 	8 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279184/fs/1265800/fg/1952082


Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 00:01 | Remaining Time: 00:00


2026-01-19 11:42:11,574 WARNING: UserWarning: Materialization job is already running, aborting new execution.Please wait for the current execution to finish before triggering a new one.You can check the status of the current execution using `fg.materialization_job.get_state()`.or `fg.materialization_job.get_final_state()` or check it out in the Hopsworks UI.at https://c.app.hopsworks.ai:443/p/1279184/jobs/named/air_quality_1_offline_fg_materialization.
Use fg.materialization_job.run(args=-op offline_fg_materialization -path hdfs:///Projects/kristina_titanic/Resources/jobs/air_quality_1_offline_fg_materialization/config_1768459798660) to trigger the materialization job again.

✅ Inserted 1 rows for 2026-01-17
2026-01-19 11:42:11,948 INFO: 	8 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279184/fs/1265800/fg/1952082


Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 00:01 | Remaining Time: 00:00


2026-01-19 11:42:20,944 WARNING: UserWarning: Materialization job is already running, aborting new execution.Please wait for the current execution to finish before triggering a new one.You can check the status of the current execution using `fg.materialization_job.get_state()`.or `fg.materialization_job.get_final_state()` or check it out in the Hopsworks UI.at https://c.app.hopsworks.ai:443/p/1279184/jobs/named/air_quality_1_offline_fg_materialization.
Use fg.materialization_job.run(args=-op offline_fg_materialization -path hdfs:///Projects/kristina_titanic/Resources/jobs/air_quality_1_offline_fg_materialization/config_1768459798660) to trigger the materialization job again.

✅ Inserted 1 rows for 2026-01-18
2026-01-19 11:42:21,225 INFO: 	8 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279184/fs/1265800/fg/1952082


Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 00:01 | Remaining Time: 00:00


2026-01-19 11:42:29,514 WARNING: UserWarning: Materialization job is already running, aborting new execution.Please wait for the current execution to finish before triggering a new one.You can check the status of the current execution using `fg.materialization_job.get_state()`.or `fg.materialization_job.get_final_state()` or check it out in the Hopsworks UI.at https://c.app.hopsworks.ai:443/p/1279184/jobs/named/air_quality_1_offline_fg_materialization.
Use fg.materialization_job.run(args=-op offline_fg_materialization -path hdfs:///Projects/kristina_titanic/Resources/jobs/air_quality_1_offline_fg_materialization/config_1768459798660) to trigger the materialization job again.

✅ Inserted 1 rows for 2026-01-19


### 2.6.2. Verify Air Quality Insertion

In [34]:
print(all_aq[all_aq["date"].dt.date == today][["sensor_id", "date", "pm25", "pm25_lag_1d", "pm25_rolling_3d", "pm25_nearby_avg"]])

     sensor_id       date  pm25  pm25_lag_1d  pm25_rolling_3d  pm25_nearby_avg
10       57421 2026-01-19  34.0         34.0             34.0        45.333333
15       58666 2026-01-19   7.0          7.0              7.0              NaN
20       58909 2026-01-19  52.0         52.0             52.0              NaN
25       58912 2026-01-19  33.0         33.0             33.0              NaN
30       58921 2026-01-19  14.0         14.0             14.0              NaN
..         ...        ...   ...          ...              ...              ...
500     494275 2026-01-19   3.0          3.0              3.0              NaN
505     497266 2026-01-19   3.0          3.0              3.0              NaN
510     533086 2026-01-19  52.0         52.0             52.0              NaN
515     556792 2026-01-19  56.0         56.0             56.0              NaN
520     562600 2026-01-19  52.0         52.0             52.0              NaN

[103 rows x 6 columns]


In [35]:
print(all_aq[all_aq["date"].dt.date == today - timedelta(days=1)])

     sensor_id       date  pm25  pm25_lag_1d  pm25_lag_2d  pm25_lag_3d  \
9        57421 2026-01-18  34.0         34.0         34.0         34.0   
14       58666 2026-01-18   7.0          7.0          7.0          7.0   
19       58909 2026-01-18  52.0         52.0         52.0         52.0   
24       58912 2026-01-18  33.0         33.0         33.0         33.0   
29       58921 2026-01-18  14.0         14.0         14.0         14.0   
..         ...        ...   ...          ...          ...          ...   
499     494275 2026-01-18   3.0          3.0          3.0          3.0   
504     497266 2026-01-18   3.0          3.0          3.0          3.0   
509     533086 2026-01-18  52.0         52.0         52.0         52.0   
514     556792 2026-01-18  56.0         56.0         56.0         56.0   
519     562600 2026-01-18  52.0         52.0         52.0         52.0   

     pm25_rolling_3d  pm25_nearby_avg                        city  \
9               34.0        45.333333     

### 2.6.3. Batch Insert Weather Forecast Data

In [36]:
if not all_weather.empty:
    # Convert types to match feature group schema
    all_weather = all_weather.astype({
        "sensor_id": "int32",
        "temperature_2m_mean": "float64",
        "precipitation_sum": "float64",
        "wind_speed_10m_max": "float64",
        "wind_direction_10m_dominant": "float64",
    })
    
    # Ensure correct column order
    weather_fg_columns = [f.name for f in weather_fg.features]
    all_weather = all_weather[weather_fg_columns]
    
    # Insert in smaller batches to avoid connection issues
    batch_size = 100
    total_inserted = 0
    
    for i in range(0, len(all_weather), batch_size):
        batch = all_weather.iloc[i:i+batch_size]
        max_retries = 3
        
        for attempt in range(max_retries):
            try:
                weather_fg.insert(batch)
                total_inserted += len(batch)
                print(f"✅ Weather batch {i//batch_size + 1}: {len(batch)} records (total: {total_inserted}/{len(all_weather)})")
                break
            except (ProtocolError, ConnectionError, TimeoutError, KafkaException) as e:
                if attempt < max_retries - 1:
                    wait_time = 2 ** attempt
                    print(f"⚠️ Connection error on weather batch {i//batch_size + 1}, retrying in {wait_time}s...")
                    time.sleep(wait_time)
                else:
                    print(f"❌ Failed weather batch {i//batch_size + 1}")
                    failed_file = f"{root_dir}/failed_weather_batch_{today}_{i}.csv"
                    batch.to_csv(failed_file, index=False)
                    print(f"💾 Saved to {failed_file}")
    
    print(f"🌤️ Total weather inserted: {total_inserted}/{len(all_weather)} records")
else:
    print("⚠️ No weather data to insert")

2026-01-19 11:42:29,757 INFO: 	7 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279184/fs/1265800/fg/1945998


Uploading Dataframe: 100.00% |██████████| Rows 100/100 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279184/jobs/named/weather_1_offline_fg_materialization/executions
✅ Weather batch 1: 100 records (total: 100/1133)
2026-01-19 11:42:44,317 INFO: 	7 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279184/fs/1265800/fg/1945998


Uploading Dataframe: 100.00% |██████████| Rows 100/100 | Elapsed Time: 00:01 | Remaining Time: 00:00


2026-01-19 11:42:53,058 WARNING: UserWarning: Materialization job is already running, aborting new execution.Please wait for the current execution to finish before triggering a new one.You can check the status of the current execution using `fg.materialization_job.get_state()`.or `fg.materialization_job.get_final_state()` or check it out in the Hopsworks UI.at https://c.app.hopsworks.ai:443/p/1279184/jobs/named/weather_1_offline_fg_materialization.
Use fg.materialization_job.run(args=-op offline_fg_materialization -path hdfs:///Projects/kristina_titanic/Resources/jobs/weather_1_offline_fg_materialization/config_1768459788862) to trigger the materialization job again.

✅ Weather batch 2: 100 records (total: 200/1133)
2026-01-19 11:42:53,234 INFO: 	7 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279184/fs/1265800/fg/1945998


...

✅ Weather batch 10: 100 records (total: 1000/1133)
2026-01-19 11:43:56,839 INFO: 	7 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279184/fs/1265800/fg/1945998


Uploading Dataframe: 100.00% |██████████| Rows 100/100 | Elapsed Time: 00:01 | Remaining Time: 00:00


✅ Weather batch 11: 100 records (total: 1100/1133)
2026-01-19 11:44:05,858 INFO: 	7 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279184/fs/1265800/fg/1945998


Uploading Dataframe: 100.00% |██████████| Rows 33/33 | Elapsed Time: 00:00 | Remaining Time: 00:00


✅ Weather batch 12: 33 records (total: 1133/1133)
🌤️ Total weather inserted: 1133/1133 records


### 2.6.4. Print Processing Summary

In [37]:
# print(f"\n📊 Summary: ✅ {successful} successful, ⏭️ {skipped} skipped, ❌ {failed} failed")

## 2.7. Inspect Inserted Data

In [38]:
if 'all_aq' in locals() and not all_aq.empty:
    print(f"✅ Air quality records inserted: {len(all_aq)}")
    print("\n📋 Sample air quality data:")
    print(all_aq.head())
    print("\n🔧 Air quality data types:")
    print(all_aq.dtypes)
    print("\n📅 Date range:")
    print(f"From {all_aq['date'].min()} to {all_aq['date'].max()}")

if 'all_weather' in locals() and not all_weather.empty:
    print(f"\n🌤️ Weather records inserted: {len(all_weather)}")
    print("\n📋 Sample weather data:")
    print(all_weather.head())
    print("\n🔧 Weather data types:")
    print(all_weather.dtypes)
    print("\n📅 Unique weather dates:")
    print(all_weather['date'].unique())

✅ Air quality records inserted: 521

📋 Sample air quality data:
   sensor_id       date  pm25  pm25_lag_1d  pm25_lag_2d  pm25_lag_3d  \
0      57421 2026-01-12   4.0          NaN          NaN          NaN   
1      57421 2026-01-12   4.0          4.0          NaN          NaN   
2      57421 2026-01-13   4.0          4.0          4.0          NaN   
3      57421 2026-01-13   4.0          4.0          4.0          4.0   
4      57421 2026-01-14   4.0          4.0          4.0          4.0   

   pm25_rolling_3d  pm25_nearby_avg         city street country  \
0              NaN              NaN  Johannehill   Ubby  Sweden   
1              4.0              NaN  Johannehill   Ubby  Sweden   
2              4.0              NaN  Johannehill   Ubby  Sweden   
3              4.0              NaN  Johannehill   Ubby  Sweden   
4              4.0              NaN  Johannehill   Ubby  Sweden   

                            aqicn_url  latitude  longitude  
0  https://api.waqi.info/feed/A57421/  